# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import sys
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
import random
import os
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score,f1_score, fbeta_score, classification_report, make_scorer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('df',engine)
df.dropna(how='any', inplace =True)
X = df['message']
y = df.iloc[:,4:]
categories = y.columns

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
     # get tokens from text
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # clean tokens
    clean_tokens = []
    for token in tokens:
        token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(token)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
random.seed(20181022)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = model.predict(X_test)

In [7]:
overall_accuracy = (y_pred == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 92.47% 



In [8]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

        0.0       0.36      0.21      0.27       683
        1.0       0.67      0.82      0.73      1343
        2.0       0.67      0.11      0.18        19

avg / total       0.56      0.61      0.57      2045

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

        0.0       0.67      0.87      0.76      1357
        1.0       0.37      0.16      0.22       688

avg / total       0.57      0.63      0.57      2045

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2042
        1.0       0.00      0.00      0.00         3

avg / total       1.00      1.00      1.00      2045

------------------------------------------------------

FEATURE: aid_related

    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 2), (1, 2)),
    'clf__estimator__n_estimators': [10, 100]
 
}

cv = GridSearchCV(pipeline, parameters)

In [10]:
modelb= cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
y_pred = modelb.predict(X_test)

In [12]:
overall_accuracy = (y_pred == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 92.85% 



In [13]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

        0.0       0.39      0.06      0.11       683
        1.0       0.66      0.95      0.78      1343
        2.0       0.33      0.05      0.09        19

avg / total       0.57      0.65      0.55      2045

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

        0.0       0.67      0.94      0.78      1357
        1.0       0.42      0.09      0.14       688

avg / total       0.59      0.65      0.57      2045

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2042
        1.0       0.00      0.00      0.00         3

avg / total       1.00      1.00      1.00      2045

------------------------------------------------------

FEATURE: aid_related

    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [17]:
pipeline_new = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [18]:
pipeline_new.fit(X_train, y_train)
y_pred = pipeline_new.predict(X_test)

In [28]:
parameters= {
    'features__text_pipeline__vect__ngram_range': ((1, 2), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.75, 1.0),
    #'features__text_pipeline__vect__max_features': (None, 5000),
    'features__text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [10, 100],
    #'features__transformer_weights': (
           # {'text_pipeline': 1, 'starting_verb': 0.5},
            #{'text_pipeline': 0.5, 'starting_verb': 1},
           # {'text_pipeline': 0.8, 'starting_verb': 1},
    #)
}

In [29]:
cv = GridSearchCV(pipeline_new, param_grid = parameters, verbose = 2, n_jobs = 1)
modelc = cv.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] clf__estimator__n_estimators=10, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=10, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__ngram_range=(1, 2), total=  30.4s
[CV] clf__estimator__n_estimators=10, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   42.5s remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__ngram_range=(1, 2), total=  30.4s
[CV] clf__estimator__n_estimators=10, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=10, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__ngram_range=(1, 2), total=  30.7s
[CV] clf__estimator__n_estimators=10, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=10, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__ngram_range=(1, 2), total=  30.8s
[CV] clf__estimator__n_estimators=10, features__text_pipelin

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 79.4min finished


In [30]:
y_pred = modelc.predict(X_test)

In [31]:
overall_accuracy = (y_pred == y_test).mean().mean()
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 92.88% 



In [32]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATUlRE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATUlRE: related

             precision    recall  f1-score   support

        0.0       0.33      0.00      0.00       683
        1.0       0.66      1.00      0.79      1343
        2.0       0.00      0.00      0.00        19

avg / total       0.54      0.66      0.52      2045

------------------------------------------------------

FEATUlRE: request

             precision    recall  f1-score   support

        0.0       0.68      0.92      0.78      1357
        1.0       0.46      0.14      0.21       688

avg / total       0.60      0.66      0.59      2045

------------------------------------------------------

FEATUlRE: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2042
        1.0       0.00      0.00      0.00         3

avg / total       1.00      1.00      1.00      2045

------------------------------------------------------

FEATUlRE: aid_related



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [33]:
filename = 'classifier.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.